## COVID19 - Shelter Layout and Behavior
Here we will simulate a shelter with a particular layout for living spaces and a number of evacuees with daily activities. We will measure the risk of infection under various conditions.

### Instruccions
<a id=start></a>
1. [Setting](#setting)
2. [Setup](#setup)
3. [Run](#run)

### 3. RUN THE MODEL
<a id=run></a>
Here is the main [Model](#model) if you want to take a look.

In [ ]:
runs = 10#5000
inf_df = np.zeros(runs)
place_df = np.zeros((runs,4))
for run in range(runs):
    setup(pop=number_of_evacuees,sim_time=number_of_days,num_inf=number_of_initial_infected,
          num_pods=number_of_pods,cap_ca=capacity_of_common_areas,cap_pod=capacity_in_pods,
          files=False,plots=False,verbose=False)
    go(files=False,plots=False,videos=False,verbose=False)
    place_df[run,:] = e_place_of_infection
    inf_df[run] = e_system_log[e_day*24+e_time,2]
    #evaluate convergence
#     if run >= 2:
#         cv2 = inf_df[:run].std() / inf_df[:run].mean()
#         cv1 = inf_df[:run-1].std() / inf_df[:run-1].mean()
#         print(run,abs(cv2-cv1))
#         if abs(cv2-cv1) < 0.0000001:
#             inf_df = inf_df[:run]
#             place_df = place_df[:run]
#             break
#     print(f'Infected:{e_system_log[-1,2]}')
#     print(f'Place of Infection(H,T,M,F):{e_place_of_infection}')

In [ ]:
inf_df.mean()

In [ ]:
y = []
for m in range(2,len(inf_df)):
#     y.append(inf_df[:m].std() / inf_df[:m].mean())
#     y.append(inf_df[:m].std())
    y.append(inf_df[:m].mean())

In [ ]:
# np.savetxt(f"baseline_infected5000.csv",inf_df[:],delimiter=',',header="Tinf",comments='',fmt='%d')
np.savetxt(f"baseline2_places5000.csv",place_df[:,:],delimiter=',',
            header="home,toilet,meeting,foodcourt",comments='',fmt='%d,%d,%d,%d')

In [ ]:
fig, ax = plt.subplots()
x = np.linspace(0,len(inf_df)-3,len(inf_df)-2)
# ax.plot(x,inf_df,'r-',label='total')
# ax.plot(x,y,'r-',label='Std')
ax.plot(x,y,'r-',label='Sample Mean')
# ax.plot(x,place_df.transpose()[0],'g--',label='home')
# ax.plot(x,place_df.transpose()[1],'b--',label='toilet')
# ax.plot(x,place_df.transpose()[2],'k--',label='meeting room')
# ax.plot(x,place_df.transpose()[3],'m--',label='food court')
ax.set_xlabel('Runs')
# ax.set_ylabel('Sample Standard Deviation (pers.)')
ax.set_ylabel('Sample Mean (pers.)')
# ax.set_yticks([16,17,18])
# ax.set_yticks([6.,6.5,7.])
plt.legend(loc='upper right');
# plt.ylim(6,7)
# plt.ylim(0.35,0.45)
# plt.savefig('Baseline.png',dpi=300)

In [ ]:
import pandas as pd

pdf = pd.read_csv('baseline2_places5000.csv')
sizes = pdf.mean()

In [ ]:
# inf_df = pdf.sum(axis=1)
sizes

In [ ]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = ['Home', 'Toilet', 'Meeting Room', 'Food Court']
sum_of_rows = sizes.sum()
norm_sizes = sizes / sum_of_rows
sizes = norm_sizes
explode = (0.1, 0, 0, 0)  # only "explode" the 1st slice (i.e. 'Home')

fig, ax = plt.subplots()
ax.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90,normalize=False)
ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
# plt.savefig('baseline_pie.png',dpi=300)

---

### 1. PARAMETER SETTING
<a id=setting></a>

In [ ]:
erase_case = True
case = "Baseline_2"
fix_random = True
number_of_evacuees = 132
number_of_days = 50
number_of_initial_infected = 1

#management
do_isolate = False     #to perform isolation after a symptomatic appears
number_of_pods = 20
capacity_of_common_areas = [4,25,25] #toilet,meeting,foodcourt
capacity_in_pods = 4

#behavioral countermeasures
rate_of_mask_use = 1. #rate of people using masks
disinfect_every_n_hours = 0 #to promote disinfection every n hours. Assuming it goes after 1 hour.

[Go to Start](#start)

> AFTER SETTING PARAMETERS 'RUN BELOW' FROM HERE

### 2. SETUP FROM HERE
<a id=setup></a>

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import pylab as plt
import matplotlib as mpl
from matplotlib.ticker import FormatStrFormatter
# params = {'text.usetex': True}
# plt.rcParams.update(params)
import os
import cv2
import time
import glob, os, os.path
import shutil

In [ ]:
#create case folders
if erase_case:
    if os.path.exists(f'./{case}'):
        shutil.rmtree(f'./{case}')
os.mkdir(f'./{case}')
os.mkdir(f'./{case}/images_ca')
os.mkdir(f'./{case}/images_hp')
os.mkdir(f'./{case}/images_c')
os.mkdir(f'./{case}/schedules')
os.mkdir(f'./{case}/densities_hp')
os.mkdir(f'./{case}/densities_ca')
os.mkdir(f'./{case}/logs')

In [ ]:
#erase files
# filelist1 = glob.glob(os.path.join('./images_ca', "*.png"))
# filelist2 = glob.glob(os.path.join('./images_hp', "*.png"))
# filelist3 = glob.glob(os.path.join('./images_c', "*.png"))
# filelist4 = glob.glob(os.path.join('./schedules', "*.png"))
# filelist5 = glob.glob(os.path.join('./densities_hp', "*.csv"))
# filelist6 = glob.glob(os.path.join('./densities_ca', "*.csv"))
# filelist7 = glob.glob(os.path.join('./logs', "*.csv"))
# filelist8 = glob.glob(os.path.join('.', "*.avi"))
# filelist9 = glob.glob(os.path.join('.', "*.png"))
# L = filelist1+filelist2+filelist3+filelist4+filelist5+filelist6+filelist7+filelist8+filelist9
# for f in L:
#     os.remove(f)

In [ ]:
if fix_random:
    np.random.seed(0)
e_day = 0
e_time = 0
e_pop = number_of_evacuees #number of evacuees in shelter
e_sim_time = number_of_days #scale in days
e_rng = np.random #np.random.default_rng()
e_list_of_infected = [] #current
e_list_of_symptomatics = []
e_list_of_isolated = []
e_place_of_infection = np.zeros(4)
e_patient_zero = number_of_initial_infected
e_number_of_infected = 0 #total (incluidng isolated)
e_number_of_symptomatics = 0
e_number_of_isolated = 0
e_use_of_mask = rate_of_mask_use + 0.001 #to avoid division by zero
e_freq_disinfection = disinfect_every_n_hours #to avoid zero
e_num_of_pods = number_of_pods
e_capacity_of_common_areas = capacity_of_common_areas #toilet,meeting,foodcourt
e_density_in_common_areas = np.zeros((3,3,e_sim_time*24)) #id,num_evac,den
for i in range(e_sim_time*24):
    e_density_in_common_areas[:,0,i]= np.linspace(0,2,3).astype(int)
e_capacity_of_pods = [capacity_in_pods]*e_num_of_pods
e_density_in_pods = np.zeros((e_num_of_pods,3,e_sim_time*24)) #id,num_evac,den
for i in range(e_sim_time*24):
    e_density_in_pods[:,0,i]= np.linspace(0,e_num_of_pods-1,e_num_of_pods).astype(int)
e_max_num_fam = 0
#age,home,schedule
e_day_log = np.zeros((e_pop,26,e_sim_time)).astype(int)
#day,hour,e_number_of_infected,len(e_list_of_infected),e_number_of_symptomatics,e_number_of_isolated
e_system_log = np.zeros((e_sim_time*24,6)).astype(int)
# for i in range(e_sim_time*24):
#     e_system_log[i,0]= int(i)
#day,hour,home,den,mask,disinf,inf,inc,sym,out,currentloc
e_hour_log = np.zeros((e_pop,10,e_sim_time*24)).astype(int)

#define the shape of the environment (i.e., its states)
#The only things the manager can see
# state_vector = np.array([e_day,e_number_of_symptomatics,e_number_of_isolated])

#define actions
#numeric action codes: 0 = none, 1 = capacity in common areas, 2 = layout, capacity in pods
# actions = ['none', 'capacity', 'layout']
# actions = ['none']

#Create a 3D numpy array to hold the current Q-values for each state and action pair: Q(s, a) 
#The array contains 10000 possible states (rows); for three variables describing the environment.
#The "action" dimension consists of 5 layers that will allow us to keep track of the Q-values 
#for each possible action in each state (see next cell for a description of possible actions). 
#The value of each (state, action) pair is initialized to 0.
# q_values = np.zeros((10000,len(state_vector)+len(actions))).astype(int)
# q_values[:,len(state_vector):]=-99999

In [ ]:
# q_values[0,0:len(state_vector)]
# state_vector
#q_values.shape

In [ ]:
#Create a 2D numpy array to hold the rewards for each state.
# rewards = (-1)*e_number_of_infected
#CREATED LATER IN TD FUNCTIONS

#### Define Helper Functions

In [ ]:
#define a function that determines if the specified state is a terminal state
def is_terminal_state(verbose=False):
    if e_number_of_infected == e_number_of_isolated or e_number_of_infected == e_pop:
        if verbose: 
            print(">>>Got to a terminal state!<<<")
        return True
    else:
        return False

#define a function that will choose a random, non-terminal starting state
def get_starting_state(pop=42,sim_time=7,num_inf=1,num_pods=42,
                       cap_ca=[50,50,50],cap_pod=1,verbose=False):
    global e_rng
    global e_list_of_infected
    global e_list_of_symptomatics
    global e_list_of_isolated
    global e_place_of_infection
    global e_patient_zero
    global e_number_of_infected
    global e_number_of_symptomatics
    global e_number_of_isolated
    global e_use_of_mask
    global e_freq_disinfection
    global e_num_of_pods
    global e_capacity_of_common_areas
    global e_density_in_common_areas
    global e_capacity_of_pods
    global e_density_in_pods
    global e_max_num_fam
    global e_day
    global e_time
    global e_pop
    global e_sim_time
    global e_day_log
    global e_system_log
    global e_hour_log
    global state_vector 
    
    e_day = 0
    e_time = 0
    e_pop = number_of_evacuees #number of evacuees in shelter
    e_sim_time = number_of_days #one week, scale in days
    e_rng = np.random #np.random.default_rng()
    e_list_of_infected = [] #current
    e_list_of_symptomatics = []
    e_list_of_isolated = []
    e_place_of_infection = np.zeros(4)
    e_patient_zero = number_of_initial_infected
    e_number_of_infected = 0 #total (incluidng isolated)
    e_number_of_symptomatics = 0
    e_number_of_isolated = 0
    e_use_of_mask = rate_of_mask_use + 0.001 #to avoid division by zero
    e_freq_disinfection = disinfect_every_n_hours #to avoid zero
    e_num_of_pods = number_of_pods
    e_capacity_of_common_areas = capacity_of_common_areas #toilet,meeting,foodcourt
    e_density_in_common_areas = np.zeros((3,3,e_sim_time*24)) #id,num_evac,den
    for i in range(e_sim_time*24):
        e_density_in_common_areas[:,0,i]= np.linspace(0,2,3).astype(int)
    e_capacity_of_pods = [capacity_in_pods]*e_num_of_pods
    e_density_in_pods = np.zeros((e_num_of_pods,3,e_sim_time*24)) #id,num_evac,den
    for i in range(e_sim_time*24):
        e_density_in_pods[:,0,i]= np.linspace(0,e_num_of_pods-1,e_num_of_pods).astype(int)
    e_max_num_fam = 0
    #age,home,schedule
    e_day_log = np.zeros((e_pop,26,e_sim_time)).astype(int)
    #day,hour,e_number_of_infected,len(e_list_of_infected),e_number_of_symptomatics,e_number_of_isolated
    e_system_log = np.zeros((e_sim_time*24,6)).astype(int)
#     for i in range(e_sim_time*24):
#         e_system_log[i,0]= int(i)
    #day,hour,home,den,mask,disinf,inf,inc,sym,out,currentloc
    e_hour_log = np.zeros((e_pop,10,e_sim_time*24)).astype(int)
    #define the shape of the environment (i.e., its states)
#     state_vector = np.array([e_day,e_number_of_symptomatics,e_number_of_isolated])


    reset_agents(verbose=verbose)
    create_agents(verbose=verbose)
    set_infected_agents(verbose=verbose)
    set_using_mask(verbose=verbose)
    set_agents_schedules(verbose=verbose)
    set_agents_family(verbose=verbose)


#define an epsilon greedy algorithm that will choose which action to take next 
def get_next_action_random_selection(epsilon):
  #if a randomly chosen value between 0 and 1 is less than epsilon, 
  #then choose the most promising value from the Q-table for this state.
    global q_values
    init_value = -99999 #NEED  TO BE CHECKED
    #check if the state exists, a is the index of that state
    a = np.where(np.all(q_values[:,0:len(state_vector)] == state_vector,axis=1))[0]
    try:
        b = np.where(np.all(q_values == [0]*q_values.shape[1],axis=1))[0].min()
    except:
        b = -1
    
    if np.random.random() < epsilon:
        if a.size != 0:
#             print('1')
            return a,np.argmax(q_values[a.item(),len(state_vector):q_values.shape[1]]),np.max(q_values[a.item(),len(state_vector):q_values.shape[1]]),np.max(q_values[a.item(),len(state_vector):q_values.shape[1]])
        else:
            q_init = [init_value]*len(actions)
            if b != -1:
                q_values[b,:]=[*state_vector,*q_init]
#                 print('1a')
            else:
                q_values = np.append(q_values,[[*state_vector,*q_init]],0)
                b = q_values.shape[0]-1
#                 print('1b')
            return b,np.random.randint(len(actions)),init_value,init_value
    else: #choose a random action
        r = np.random.randint(len(actions))
        if a.size != 0:
#             print('2')
            return a,r,q_values[a.item(),r+len(state_vector)],np.max(q_values[a.item(),len(state_vector):q_values.shape[1]])
        else:
            q_init = [init_value]*len(actions)
            if b != -1:
                q_values[b,:]=[*state_vector,*q_init]
#                 print('2a')
            else:
                q_values = np.append(q_values,[[*state_vector,*q_init]],0)
                b = q_values.shape[0]-1
#                 print('2b')
            return b,r,init_value,init_value

#TRYING NOT TO SELECT RANDOM ACTIONS, INSTEAD ALWAYS NONE
#define an epsilon greedy algorithm that will choose which action to take next 
def get_next_action(epsilon):
  #if a randomly chosen value between 0 and 1 is less than epsilon, 
  #then choose the most promising value from the Q-table for this state.
    global q_values
    init_value = -99999 #NEED  TO BE CHECKED
    #check if the state exists, a is the index of that state
    try:
        a = np.where(np.all(q_values[:,0:len(state_vector)] == state_vector,axis=1))[0][0]
    except:
        a = -1
    try:
        b = np.where(np.all(q_values == [0]*q_values.shape[1],axis=1))[0].min()
    except:
        b = -1
    
    if np.random.random() < epsilon:
        if a != -1:
            return a,np.argmax(q_values[a,len(state_vector):q_values.shape[1]]),np.max(q_values[a,len(state_vector):q_values.shape[1]]),np.max(q_values[a,len(state_vector):q_values.shape[1]])
        else:
            q_init = [init_value]*len(actions)
            if b != -1:
                q_values[b,:]=[*state_vector,*q_init]
#                 print(f'1a-b:{b}')
            else:
                q_values = np.append(q_values,[[*state_vector,*q_init]],0)
                b = q_values.shape[0]-1
#                 print(f'1b-b:{b}')
            return b,np.random.randint(len(actions)),init_value,init_value
    else: #choose to do nothing
        r = 0 #np.random.randint(len(actions))
        if a != -1:
#             print('2')
            return a,r,q_values[a,r+len(state_vector)],np.max(q_values[a,len(state_vector):q_values.shape[1]])
        else:
            q_init = [init_value]*len(actions)
            if b != -1:
                q_values[b,:]=[*state_vector,*q_init]
#                 print('2a')
            else:
                q_values = np.append(q_values,[[*state_vector,*q_init]],0)
                b = q_values.shape[0]-1
#                 print('2b')
            return b,r,init_value,init_value


#define a function that will get the next behavior of agents based on the chosen action
def get_next_behavior_6(action_index):
    if actions[action_index] == 'none':
        pass
    elif actions[action_index] == 'isolate':
        ask_to_isolate_symptomatics(verbose=False)
    elif actions[action_index] == 'mask':
        ask_to_use_masks(verbose=False)
    elif actions[action_index] == 'disinfection':
        ask_to_disinfect(verbose=False)
    elif actions[action_index] == 'capacity':
        ask_to_reducecapacity(verbose=False)
    elif actions[action_index] == 'layout':
        ask_to_changelayout(verbose=False)
    return

#define a function that will get the next behavior of agents based on the chosen action
def get_next_behavior(action_index):
    if actions[action_index] == 'none':
        pass
    elif actions[action_index] == 'capacity':
        ask_to_reducecapacity(verbose=False)
    elif actions[action_index] == 'layout':
        ask_to_changelayout(verbose=False)
    return

#==========================

def reset_agents(verbose=False):
    "to destroy agents"
    if verbose: 
        print(">Reseting agents...")
    Agents.count = 0
    for evacuee in Agents.agent:
        del evacuee
    Agents.agent = []
    
def create_agents(verbose=False):
    "to create new agents"
    if verbose:
        print(f">Creating {e_pop} agents...")
    for i in range(e_pop):
        Agents(i,int(np.random.normal(40,12)))
        
def set_infected_agents(verbose=False):
    "to randomnly select number agents as infected"
    if verbose:
        print(f">Setting initial infected agents: {e_patient_zero}")
    candidates = e_rng.choice(Agents.agent,e_patient_zero)
    for evacuee in candidates:
        evacuee.a_getVirus(verbose=verbose)
        if verbose:
            print(f"{evacuee.uid}:I am patient zero")

#call every day
def write_day_log(day=0,verbose=False):
    "to save variables every day"
    global e_day_log
    for e in Agents.agent:
        e_day_log[e.uid,:,day]=[e.age,e.home,*e.schedule[:,1]]

#call every hour
def write_hour_log(day=0,hour=0,verbose=False):
    "to save variables every hour"
    global e_hour_log
    for e in Agents.agent:
        e_hour_log[e.uid,:,e_day*24+e_time]=[day,hour,e.home,e.mask,e.disinf,e.inf,e.inc,e.sym,e.out,e.currentloc]

#call every hour
def write_system_log(day=0,hour=0,verbose=False):
    "to save system variables"
    global e_system_log
    e_system_log[e_day*24+e_time,:]=[day,hour,e_number_of_infected,len(e_list_of_infected),e_number_of_symptomatics,e_number_of_isolated]
    
def set_using_mask(verbose=False):
    "to randomnly start use of mask"
    num = int(e_use_of_mask * Agents.count)
    if verbose:
        print(f">Setting {num} of evacuees with mask")
    candidates = e_rng.choice(Agents.agent,num)
    for evacuee in candidates:
        evacuee.mask = True

def set_agents_schedules(verbose=False):
    "to initiate schedules of day"
    if verbose:
        print(f">Deciding schedules for agents...")
    for evacuee in Agents.agent:
        evacuee.a_setSchedule(verbose=verbose)

def set_agents_family(verbose=False):
    "to check who is in same pod"
    if verbose:
        print(f">Verifying family members...")
    for evacuee in Agents.agent:
        evacuee.a_calculate_my_family(verbose=verbose)

#to use every hour
def calculate_densities_at_common_areas(verbose=False):
    "to calculate densities at each location"
    global e_density_in_common_areas
    if verbose:
        print(f">Calculating densities in common areas...")
    for evacuee in Agents.agent:
        if evacuee.out:
            return
        if evacuee.currentloc == 1:
            e_density_in_common_areas[0,1,e_day*24+e_time] += 1
        elif evacuee.currentloc == 2:
            e_density_in_common_areas[1,1,e_day*24+e_time] += 1
        elif evacuee.currentloc == 3:
            e_density_in_common_areas[2,1,e_day*24+e_time] += 1
        else:
            pass
    e_density_in_common_areas[:,2,e_day*24+e_time] = e_density_in_common_areas[:,1,e_day*24+e_time] / e_capacity_of_common_areas[:]

#to use every hour
def calculate_densities_at_home_pods(verbose=False):
    "to ask agents to calculate density at home if currentloc is home"
    global e_density_in_pods
    if verbose:
        print(f">Calculating densities at home pods...")
    for evacuee in Agents.agent:
        if evacuee.out:
            return
        evacuee.a_calculate_my_density_at_home(verbose=verbose)
    e_density_in_pods[:,2,e_day*24+e_time] = e_density_in_pods[:,1,e_day*24+e_time] / e_capacity_of_pods

def reset_density_at_home_pods(verbose=False):
    "to reset pod density to original"
    global e_density_in_pods
    if verbose:
        print(f">Reseting densities at home pods...")
    e_density_in_pods[:,1,e_day*24+e_time] = 0
    #calculate_densities_at_home_pods(verbose=verbose)
        
#run every hour (step)
def set_next_location(verbose=False):
    "to set the next event of each evacuee"
    if verbose:
        print(f">Setting next location...")
    for evacuee in Agents.agent:
        if evacuee.out:
            evacuee.nextloc = -1
        else:
            evacuee.a_pick_next_loc(e_time)

def move_agents_to_next_location(verbose=False):
    "to move all agent to its new location"
    if verbose:
        print(f">Moving agents to next location...")
    for evacuee in Agents.agent:
        if evacuee.out:
            evacuee.currentloc = -1
            evacuee.nextloc = -1
            return
        evacuee.a_move(verbose=verbose)

#at the end of day
def update_infected(verbose=False):
    "to update list of infected"
    global e_list_of_infected
    global e_number_of_infected
    if verbose:
        print(f">Updating infected list and incubation days...")
    for evacuee in e_list_of_infected:
        if not evacuee.inf:
            if verbose:
                print(f"Removing evacuee {evacuee.uid} with inf:{evacuee.inf}")
            e_list_of_infected.remove(evacuee)
            e_number_of_infected -=1
        else:
            evacuee.inc -= 1
            if verbose:
                print(f"@{evacuee.uid} remaining days:{evacuee.inc}")
            if evacuee.inc == 0:
                evacuee.a_symptoms()
    
    e_list_of_infected = list(set(e_list_of_infected))
    

def calculate_virus_spreading_in_common_areas(verbose=False):
    "to spread virus based on probabilities and location"
    if verbose:
        print(f">Calculating virus spread in common areas...")
    num_per_area = [0,0,0] #toilet,meeting,foodcourt
    for evacuee in e_list_of_infected:
        if evacuee.out:
            return
        if evacuee.currentloc == 0:
            return
        if evacuee.currentloc == 1: #toilet
            num_per_area[0] += 1
        if evacuee.currentloc == 2: #meeting
            num_per_area[1] += 1
        if evacuee.currentloc == 3: #foodcourt
            num_per_area[2] += 1
    
    for evacuee in Agents.agent:
        if evacuee.out or evacuee.inf:
            return
        evacuee.a_check_infection_risk_in_common_areas(num_per_area,verbose=verbose)

def calculate_virus_spreading_at_home_pods(verbose=False):
    "to spread virus based on probabilities at homes"
    if verbose:
        print(f">Calculating virus spread at homes...")
    for evacuee in e_list_of_infected:
        if evacuee.out:
            return
        if evacuee.currentloc == 0 and len(evacuee.fam) > 0:
            n_inf_mem = 1
            n_mem = len(evacuee.fam)
            for member in evacuee.fam:
                if member.inf and not member.out:
                    n_inf_mem +=1
            for member in evacuee.fam:
                if not member.inf and not member.out:
                    member.a_check_infection_risk_at_home(n_inf_mem,verbose=verbose)

def apply_disinfection(verbose=False):
    for evacuee in Agents.agent:
        evacuee.a_disinfect_apply(verbose=verbose)

def remove_disinfection(verbose=False):
    for evacuee in Agents.agent:
        evacuee.a_disinfect_remove(verbose=verbose)
        
#move clock one hour
def step(verbose=False):
    "to move one point in time"
#     global e_time
#     e_time += 1
    if verbose:
        print("="*10)

#### Functions for agents

In [ ]:
class Agents(object):
    """ An agent class"""
    count = 0
    agent = []
    def __init__(self, uid, a,verbose=False):
        "Initiate agent variables"
        self.uid = self.count
        self.__class__.count +=1
        self.age = a
        self.home = np.random.choice(e_num_of_pods,1).item()
        self.fam = []
        self.mask = False
        self.disinf = False
        self.inf = False
        self.inc = np.random.choice(np.linspace(2,14).astype(int),1).item() #days
        self.sym = False
        self.out = False
        self.currentloc = 0 #(0)home; (1)toilet; (2)meeting; (3)foodcourt
        self.schedule = np.zeros((24,2)).astype(int) #list of list [time event]
        self.nextloc = 0 #code of next event (0)home; (1)toilet; (2)meeting; (3)foodcourt
        self.currentden = 0
        Agents.agent.append(self)
                    
    def a_check_infection_risk_in_common_areas(self,num_of_infected,verbose=False):
        "to check risk based on age and punished by conditions"
        p = 0
        #basic susceptibility based on age
        if self.age < 20:
            p = 0.00006
        elif self.age >= 20 and self.age < 30:
            p=0.0003
        elif self.age >= 30 and self.age < 40:
            p=0.0008
        elif self.age >= 40 and self.age < 50:
            p=0.0015
        elif self.age >= 50 and self.age < 60:
            p=0.006
        elif self.age >= 60 and self.age < 70:
            p=0.022
        elif self.age >= 70 and self.age < 80:
            p=0.051
        else:
            p=0.093
        
        #increased or reduction based on conditions
        if self.mask:
            p = 0.5*p
        
        #based on disinfection
        if self.disinf:
            p = 0.5*p
        
        #based on location
        if self.currentloc != 0:
            p = num_of_infected[self.currentloc - 1]*p 
            #last is best since if no one infected nearby then p=0, 
            #otherwise increases with number of infected people
        
            #based on density
            p  = e_density_in_common_areas[self.currentloc - 1,2,e_day*24+e_time]*p
        
        #get infected based on probability p
        if np.random.random() < p:
            if verbose:
                print(f"@{self.uid} my prob was {p:0.2f}")
            self.a_getVirus(verbose=verbose)

    def a_check_infection_risk_at_home(self,num_of_infected,verbose=False):
        "to check risk based on age and punished by conditions"
        p = 0
        #basic susceptibility based on age
        if self.age < 20:
            p = 0.00006
        elif self.age >= 20 and self.age < 30:
            p=0.0003
        elif self.age >= 30 and self.age < 40:
            p=0.0008
        elif self.age >= 40 and self.age < 50:
            p=0.0015
        elif self.age >= 50 and self.age < 60:
            p=0.006
        elif self.age >= 60 and self.age < 70:
            p=0.022
        elif self.age >= 70 and self.age < 80:
            p=0.051
        else:
            p=0.093
        
        #increased or reduction based on conditions
        if self.mask:
            p = 0.5*p
        
        #based on disinfection
        if self.disinf:
            p = 0.5*p

        #based on location
        p = num_of_infected*p 
        
        #based on density
        p  = e_density_in_pods[self.home,1,e_day*24+e_time]*p
        
        #get infected based on probability p
        if np.random.random() < p:
            if verbose:
                print(f"@{self.uid} my prob was {p:0.2f}")
            self.a_getVirus(verbose=verbose)
    
    def a_calculate_my_family(self,verbose=False):
        "to calculate who is my family - same pod"
        global e_max_num_fam
        for evacuee in Agents.agent:
            if evacuee is not self:
                if evacuee.home == self.home:
                    self.fam.append(evacuee)
        #update max num of family
        if len(self.fam) > e_max_num_fam:
                e_max_num_fam = len(self.fam)
        if verbose:
            print(f"@{self.uid} has {len(self.fam)} members at #{self.home} pod")

    def a_calculate_my_density_at_home(self,verbose=False):
        "to calculate the density in a pod"
        global e_density_in_pods
        if self.out:
            return
        if self.currentloc == 0:
            e_density_in_pods[self.home,1,e_day*24+e_time]+=1
#         else:
#             e_density_in_pods[self.home,1,e_time]-=1
        
    def a_getVirus(self,verbose=False):
        "to get infected"
        global e_list_of_infected
        global e_number_of_infected
        global e_place_of_infection
        self.inf = True
        e_list_of_infected.append(self)
        e_number_of_infected += 1
        e_place_of_infection[self.currentloc] += 1
        if verbose:
            print(f"@{self.uid} infected at {self.currentloc}")
    
    def a_setSchedule(self,verbose=False):
        "to set today's schedule"
        if self.out:
            return
        #structure of schedule
        #7-8,12-14,18-19 breakfast (fix) ==> locations 0 or 3 (home or foodcourt)
        #9-11,15-17,20-22 random number of activities (max 3 actions) (toilet, meeting, home)
        self.schedule[:,1]=np.zeros((24,), dtype=int) #set default home
        self.schedule[:,0]=np.linspace(0,23,24,dtype=int) #set hours of day
        #meals
        c = e_rng.choice([7,8],1,p=[0.5,0.5]).item() #random time for breakfast
        self.schedule[c,1]= e_rng.choice([0,3],1,p=[0.5,0.5]).item()
        c = e_rng.choice([12,13,14],1,p=[0.4,0.3,0.3]).item() #random time for lunch
        self.schedule[c,1]= e_rng.choice([0,3],1,p=[0.5,0.5]).item()
        c = e_rng.choice([18,19],1,p=[0.5,0.5]).item() #random time for dinner
        self.schedule[c,1]= e_rng.choice([0,3],1,p=[0.5,0.5]).item()
        #other activities
        slots = [[9,10,11],[15,16,17],[20,21,22]]
        for slot in slots:
            c = e_rng.choice(slot,1,p=[1./3.,1./3.,1./3.]).item() #random time for toilet
            self.schedule[c,1]= e_rng.choice([0,1,2],1,p=[1./3.,1./3.,1./3.]).item() #maybe toilet
            slot.remove(c)
            a0 = e_rng.choice([0,2],1,p=[0.5,0.5])
            a1 = e_rng.choice([0,2],1,p=[0.5,0.5])
            self.schedule[slot[0],1]=a0
            self.schedule[slot[1],1]=a1
        if verbose:
            pass#print(f"@{self.uid}:schedule set")#{self.schedule}")
    
    def a_pick_next_loc(self,e_time,verbose=False):
        "to pick the corresponding event at time e_time"
        self.nextloc = self.schedule[e_time,1]
    
    def a_move(self,verbose=False):
        "to move to next activity"
        self.currentloc = self.nextloc
        #adjust density at home
#         self.a_calculate_my_density_at_home(verbose=False)
    
    def a_symptoms(self,verbose=False):
        "to show symptoms"
        global e_list_of_symptomatics
        global e_number_of_symptomatics
        self.sym = True
        e_list_of_symptomatics.append(self)
        e_number_of_symptomatics += 1
        if verbose:
            print(f"@{self.uid} showing symptomps!")
        
    def a_isolate(self,verbose=False):
        "to take out of simulation"
        global e_list_of_infected
        global e_list_of_isolated
        global e_number_of_isolated
        global e_list_of_symptomatics
        global e_number_of_symptomatics
        self.out = True
        self.inc = -1
        self.currentloc = -1
        self.schedule[:,1] = [-1]*24
        e_list_of_infected.remove(self)
        e_list_of_symptomatics.remove(self)
        e_number_of_symptomatics -= 1
        e_list_of_isolated.append(self)
        e_number_of_isolated += 1
        if verbose:
            print(f"@{self.uid} isolated!")
        
    def a_disinfect_apply(self,verbose=False):
        self.disinf = True
            
    def a_disinfect_remove(self,verbose=False):
        self.disinf = False

#### Functions to adjust evacuee behavior (asked by manager)

In [ ]:
def ask_to_isolate_symptomatics(verbose=False):
    global e_list_of_infected
    global e_number_of_infected
    if verbose:
        print(f"!Asking symptomatics to isolate")
    for evacuee in Agents.agent:
        if evacuee.inf and evacuee.sym and not evacuee.out:
            evacuee.a_isolate(verbose=verbose)       

def ask_to_use_masks(verbose=False):
    "Increasing on p=20% the use of masks"
    global e_use_of_mask
    
    p = 0.2 #<=== include later as parameter
    if verbose:
        print(f"!Asking to increase use of masks")
    #list up people not using masks
    no_mask = [evacuee for evacuee in Agents.agent if not evacuee.mask]
    rate = len(no_mask) / e_pop
    if rate < p:
        #all
        candidates = no_mask
        e_use_of_mask += rate
        if verbose:
            print(f">(a) Setting additional {rate}({len(no_mask)}) evacuees with mask")
    else:
        #some
        num = int(p*len(no_mask))
        candidates = e_rng.choice(no_mask,num)
        e_use_of_mask += p
        if verbose:
            print(f">Setting additional {p*100}%({num}) evacuees with mask")
    
    for evacuee in candidates:
        evacuee.mask = True

def ask_to_disinfect(verbose=False):
    "when disinfecting frequency is reduced"
    global e_freq_disinfection
    e_freq_disinfection -= 1
    if e_freq_disinfection == 0:
        e_freq_disinfection = 1

def ask_to_reducecapacity(verbose=False):
    "to reduce capacity of common areas"
    global e_capacity_of_common_areas
    e_capacity_of_common_areas = list(np.array(e_capacity_of_common_areas)-1)
    e_capacity_of_common_areas = [1 if i < 1 else i for i in e_capacity_of_common_areas ]
    
def ask_to_changelayout(verbose=False):
    "to increase capacity of pods"
    global e_capacity_of_pods
    e_capacity_of_pods = list(np.array(e_capacity_of_pods)+1)

#### Functions to plot outputs

In [ ]:
def analyze_data(verbose=False):
    dfA = pd.read_csv("daylog.csv")
    dfB = pd.read_csv("hourlog.csv")
    dfC = pd.read_csv("systemlog.csv")
    return dfA, dfB, dfC

def plot_histogram():
    #plot age histogram
    mu, sigma = 40, 12
    count, bins, ignored = plt.hist(e_day_log[:,0,0],30,density=True)
    plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) *
                   np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
             linewidth=2, color='r')
    plt.xlim(0,90)
    plt.xlabel("age")
    plt.ylabel("frequency")
    plt.title("Histogram of Population age")
    plt.text(10,0.03,r"$\mathcal{N}(40,12)$",fontsize=12)
    plt.savefig(f"./{case}/histogram.png",dpi=150,facecolor="w",transparent=False)
    plt.close('all')

def plot_home_pods(day=e_day,hour=e_time,cmap="Reds",gridcolor='black'):
    #plot homes
    fig, ax = plt.subplots(figsize=(14,12))
    a = e_density_in_pods[:,2,e_time]
    a = a.reshape(5,4) #<==== hard coded :(
    plt.imshow(a,cmap=cmap)
    
    #Mayor ticks
    ax.set_xticks(np.arange(-.5, 4, 1), minor=False)
    ax.set_yticks(np.arange(-.5, 5, 1), minor=False)
    ax.grid(which='major', color=gridcolor, linestyle='-', linewidth=6)
    # Minor ticks
    ax.set_xticks(np.arange(-.5, 4, 1), minor=True)
    ax.set_yticks(np.arange(-.5, 5, 1), minor=True)
    ax.grid(which='minor', color=gridcolor, linestyle='-', linewidth=6)
    
    #hide labels
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    
    # Loop over data dimensions and create text annotations.
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            text = ax.text(j, i, a[i, j], fontsize=20,backgroundcolor="white",
                           ha="center", va="center", color="black")
    ax.set_title("Situation of home pods",fontsize=20)
    
    fig.text(0.45,0.1,f"DAY:{day} - TIME:{e_time}:00",fontsize=20,ha="left")
        
    t = 24*day+hour
    plt.savefig(f"./{case}/images_hp/homepod{t:04}.png",dpi=150,facecolor="w",transparent=False)
    fig = None
    ax = None
    plt.close('all')

def plot_common_areas(day=e_day,hour=e_time,cmap="Reds",gridcolor="black"):
    #plot places
    fig, ax =  plt.subplots()
    b = e_density_in_common_areas[:,2,e_day*24+e_time]
    b = b.reshape(1,3)
    plt.imshow(b,cmap=cmap)
    
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    #Mayor ticks
    ax.set_xticks(np.arange(-.5, 3, 1), minor=False)
    ax.set_yticks(np.arange(-.5, 1, 1), minor=False)
    ax.grid(which='major', color=gridcolor, linestyle='-', linewidth=6)
    
    #Text in places
    ax.text(0,0.5,"Toilet",fontsize=14,backgroundcolor="white",ha="center")
    ax.text(1,0.5,"Meeting room",fontsize=14,backgroundcolor="white",ha="center")
    ax.text(2,0.5,"Foodcourt",fontsize=14,backgroundcolor="white",ha="center")
    # Loop over data dimensions and create text annotations.
    for i in range(b.shape[0]):
        for j in range(b.shape[1]):
            text = ax.text(j, i, b[i, j], fontsize=20,backgroundcolor="white",
                           ha="center", va="center", color="black")
    ax.set_title("Situation of service areas")
    
    fig.text(0.4,0.15,f"DAY:{day} - TIME:{e_time}:00",fontsize=10,ha="left")
    
    t = 24*day+hour
    plt.savefig(f"./{case}/images_ca/commonarea{t:04}.png",dpi=150,facecolor="w",transparent=False)
    fig = None
    ax = None
    plt.close('all')

def plot_curves(day=e_day,hour=e_time):
    if day*24+hour == 0:
        return "Snapshot 0 is ignored"
    #plot system results
    fig, ax =  plt.subplots()

    t= day*24+hour
    ax.plot(e_system_log[:t,2],label="total infected",ls='-',c='r')
    ax.plot(e_system_log[:t,3],label="current infected",ls=':',c='orange')
    ax.plot(e_system_log[:t,4],label="symptomatics",ls='-.',c='m')
    ax.plot(e_system_log[:t,5],label="isolated",ls='--',c='k')
    ymax = e_system_log[:t,2].max() * 1.05
    ax.set_ylim(0,ymax)
    ax.yaxis.set_major_formatter(FormatStrFormatter('%0.0f'))
    ax.set_xlabel("time (hr)")
    ax.set_ylabel("Population")
    ax.set_title("Summary of shelter situation")
    ax.legend(loc='best')

    fig.text(0.4,0.15,f"DAY:{day} - TIME:{e_time}:00",fontsize=10,ha="left")
    
    plt.savefig(f"./{case}/images_c/curves{t:04}.png",dpi=150,facecolor="w",transparent=False)
    fig = None
    ax = None
    plt.close('all')
    
def get_snapshot(day=e_day,e_time=e_time):
    plot_home_pods(day,e_time)
    plot_common_areas(day,e_time)
    plot_curves(day,e_time)

def plot_schedules(day=e_day,verbose=False):
    fig, (ax1,ax2) = plt.subplots(1,2,gridspec_kw={'width_ratios': [35, 1]},figsize=(10,6))

    s = e_day_log[:,2:,day]
    s = np.rot90(s,k=3)
    
    cmap = mpl.colors.ListedColormap(['white','green','blue','red'])
    bounds=[0,0.9,1.9,2.9,3.9]
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    ax1.imshow(s,cmap=cmap, norm=norm,aspect='auto',origin="lower")
    ax1.tick_params(axis='x',labelsize=12)
    ax1.tick_params(axis='y',labelsize=12)
    #Mayor ticks
    ax1.set_xticks(np.arange(-.5,e_pop, 1), minor=True)
    ax1.set_yticks(np.arange(-.5, 23, 1), minor=True)
    ax1.grid(which='minor', color='grey', linestyle='-', linewidth=1)
    ax1.set_xticks(np.linspace(0,e_pop,11).astype(int))
    ax1.set_yticks(np.linspace(0,23,24).astype(int))
    ax1.set_xlabel("Evacuee id",fontsize=12)
    ax1.set_ylabel("time (hr)",fontsize=12)
    ax1.grid(False)
    # make a color bar
    cb = mpl.colorbar.ColorbarBase(ax2,cmap=cmap,norm=norm,ticks=[0.5,1.5,2.5,3.5],orientation='vertical')
    cb.set_ticklabels(["Home","Toilet","Meeting","Food"])
    cb.ax.tick_params(labelsize='large')
    
    fig.text(0.0,0.01,f"DAY:{day}",fontsize=20,backgroundcolor="white",ha="left")
        
    name = f"./{case}/schedules/{day:04}.png"
    fig.savefig(name,dpi=150)
    fig = None
    ax1 = None
    ax2 = None
    plt.close('all')

In [ ]:
def makeVideo(image_folder='images_c',video_name = 'video_c.avi',fps=10,verbose=False):
    images = [img for img in sorted(os.listdir(image_folder)) if img.endswith(".png")]
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape
    video = cv2.VideoWriter(video_name,cv2.VideoWriter_fourcc('M','J','P','G'), fps, (width,height))
    for image in images:
        if verbose:
            print(image)
        video.write(cv2.imread(os.path.join(image_folder, image)))
    cv2.destroyAllWindows()
    video.release()

## The Model
<a id=model></a>

In [ ]:
def setup(pop=132,sim_time=50,num_inf=1,num_pods=20,cap_ca=[4,25,25],
          cap_pod=4,files=False,plots=False,verbose=False):
    get_starting_state(pop=pop,sim_time=sim_time,num_inf=num_inf,num_pods=num_pods,
                       cap_ca=cap_ca,cap_pod=cap_pod,verbose=verbose)
    write_day_log(0,verbose=verbose)
    if files:
        np.savetxt(f"./{case}/logs/daylog-0000.csv",e_day_log[:,:,e_time],delimiter=',',
               header="age,home,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23",comments='',fmt='%26d')
    if plots:
        plot_histogram()

#setup(e_pop=132,e_sim_time=50,num_inf=1,num_pods=20,cap_ca=[4,25,25],
 #         cap_pod=4,verbose=False):

In [ ]:
def go(files=False,plots=False,videos=False,verbose=False):
    global e_day
    global e_time
    t0 = time.time()
    for i in range(e_sim_time):
        e_day = i
        set_agents_schedules(verbose=verbose)
        for j in range(24):
            e_time = j
            if verbose:
                print(f"Day {e_day}, {e_time} hours")
            if e_freq_disinfection != 0:
                if e_time % e_freq_disinfection == 0: #every 1-6 hours (compulsory disinfection at beginning of day)
                    apply_disinfection(verbose=verbose)
            set_next_location(verbose=verbose)
            move_agents_to_next_location(verbose=verbose)
            calculate_densities_at_common_areas(verbose=verbose)
            calculate_densities_at_home_pods(verbose=verbose)
            calculate_virus_spreading_in_common_areas(verbose=verbose)
            calculate_virus_spreading_at_home_pods(verbose=verbose)
            remove_disinfection(verbose=verbose)
            write_hour_log(e_day,e_time,verbose=verbose)
            write_system_log(e_day,e_time,verbose=verbose)
            if files:
                np.savetxt(f"./{case}/densities_hp/den{24*e_day+e_time:04}.csv",e_density_in_pods[:,:,e_time],delimiter=',',comments='',
                           fmt=['%d','%d','%0.2f'])
                np.savetxt(f"./{case}/densities_ca/den{24*e_day+e_time:04}.csv",e_density_in_common_areas[:,:,e_day*24+e_time],delimiter=',',
                           comments='',fmt=['%0.2f','%0.2f','%0.2f'])
                np.savetxt(f"./{case}/logs/hourlog{24*e_day+e_time:04}.csv",e_hour_log[:,:,e_time],delimiter=',',
                           header="day,hour,home,mask,disinf,inf,inc,sym,out,currentloc",comments='',fmt='%10d')
                np.savetxt(f"./{case}/logs/systemlog{24*e_day+e_time:04}.csv",e_system_log[:,:],delimiter=',',
                            header="day,hour,Tinf,Cinf,sym,isol",comments='',fmt='%6d')
            if plots:
                get_snapshot(e_day,e_time)
            step(verbose=verbose)
        update_infected(verbose=verbose)
        write_day_log(i,verbose=verbose)
        if files:
            np.savetxt(f"./{case}/logs/daylog{e_day:04}.csv",e_day_log[:,:,i],delimiter=',',
                       header="age,home,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23",comments='',fmt='%26d')
        if plots:
            plot_schedules(i,verbose=verbose)
        if verbose:
            print("*"*25)
            print(f"Total of infected evacs:{e_number_of_infected}")
            print(f"Current infected (today):{len(e_list_of_infected)}")
            print(f"Current isolated (symp):{e_number_of_infected - len(e_list_of_infected)}")
            print("*"*25)
            print("*"*25)
            print(f"Total of infected evacs:{e_number_of_infected}")
            print(f"Current infected (today):{len(e_list_of_infected)}")
            print(f"Current symptomatics (symp):{e_number_of_symptomatics}")
            print(f"Current isolated (symp):{e_number_of_isolated}")
            print("*"*25)
        #evaluate and perform countermeasures for next day
        if do_isolate:
            ask_to_isolate_symptomatics(verbose=verbose)
        #ask_to_disinfect(verbose=verbose)
        #ask_to_use_masks(verbose=verbose)
        #ask_to_reducecapacity(verbose=verbose)
        #ask_to_changelayout(verbose=verbose)
        e_time =  0 #restart day clock
        #reset_density_at_home_pods(verbose=verbose)
        if is_terminal_state(verbose=verbose):
            break
#     if plots:
#         get_snapshot(e_sim_time,23)
    if videos:
        makeVideo(image_folder=f'./{case}/schedules',video_name = f'./{case}/schedules.avi',fps=1,verbose=verbose)
        makeVideo(image_folder=f'./{case}/images_c',video_name = f'./{case}/curves.avi',fps=10,verbose=verbose)
        makeVideo(image_folder=f'./{case}/images_ca',video_name = f'./{case}/commonareas.avi',fps=5,verbose=verbose)
        makeVideo(image_folder=f'./{case}/images_hp',video_name = f'./{case}/homepods.avi',fps=5,verbose=verbose)
    t1 = time.time()
    print(f"Finished:{t1-t0} s")

# go()
#go(verbose=True) #only text
#go(plots=True,videos=False,verbose=False) #only plots
# go(plots=True,videos=True,verbose=False) #plots and videos
# go(plots=True,videos=True,verbose=True) #all

In [ ]:
if True:
    pass

[Go to Start](#start)

---

### Useful References

Lee, J.-S., Filatova, T., Ligmann-Zielinska, A., Hassani-Mahmooei, B., Stonedahl, F., Lorscheid, I., Voinov, A., Polhill, G., Sun, Z., and Parker, D. C. (2015). The Complexities of Agent-Based Modeling Output Analysis. Journal of Artificial Societies and Social Simulation, 18(4). https://doi.org/10.18564/jasss.2897